### Base Game

###### Omitted corner cases: dealing with yellows when a guess has 3 of the same letter. But reasonably no one should waste a guess like that

In [1]:
class WordleGame:
    def __init__(self, word):
        self.word = word.lower()
        self.correct_letter_pos = {}
        self.number_of_correct_letter_pos = 0
        
    def check_right_letter(self, letter):
        if letter in self.word:
            return True
        
    def check_right_position(self, letter, i):
        if letter == self.word[i]:
            return True
    
    def is_isogram(self, string):
        for i in string:
            if string.count(i) > 1:
                return False
        return True
    
    def check_all(self, guess):
        guess = guess.lower()
        self.correct_letter_pos = {}
        self.number_of_correct_letter_pos = 0
        
        for i in range(5):
            if self.check_right_letter(guess[i]) == True:
                if self.check_right_position(guess[i], i) == True:
                    self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Green'
                    self.number_of_correct_letter_pos += 1   
                
                # Yellows are quite complicated. Need to consider a few cases when we have a right letter but not right position:                
                else: 
                    # if there is only one of that letter in our guess, we set it to yellow  
                    if sum(char == guess[i] for char in guess) == 1:
                        self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Yellow'
                        
                    # if there are two of that letter in our guess, we consider a few cases:
                    elif sum(char == guess[i] for char in guess) == 2:
                        
                        # 1. The word only has one of that letter.
                        if sum(char == guess[i] for char in self.word) == 1:
                            # If the letter has not yet appeared, we set it to yellow. If not, we set it to black.
                            # Additionally, if that letter is in a correct position somewhere down the line, we need to set the incorrect position one to black as well
                            if guess[i] not in guess[:i]: 
                                self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Yellow'
                                for k in range(5):
                                    if self.check_right_position(guess[k], k) == True:
                                        if guess[k] == guess[i]:
                                            self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Black'
                            else:
                                self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Black'
                                      
                        # 2. The word has two or more of that character, set our character to yellow in all cases.
                        else:
                            self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Yellow' 
                            
                    elif sum(char == guess[i] for char in guess) == 3:
                        self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Yellow'
                        
       
            else:
                self.correct_letter_pos[str(i) + ' ' + str(guess[i])] = 'Black'
                
        return(self.correct_letter_pos) 
        
    
# will omit correcting the corner case where you guess 3 of the same letters, but let's assume no rational person will not guess that

In [2]:
# testing
word = 'CYNIC'

guess1 = 'CRANE'
guess2 = 'POUND'
guess3 = 'CACTI'
guess4 = 'CYNIC'

wrd = WordleGame(word)

print(wrd.check_all(guess1))
print(wrd.check_all(guess2))
print(wrd.check_all(guess3))
print(wrd.check_all(guess4))

wrd.number_of_correct_letter_pos

{'0 c': 'Green', '1 r': 'Black', '2 a': 'Black', '3 n': 'Yellow', '4 e': 'Black'}
{'0 p': 'Black', '1 o': 'Black', '2 u': 'Black', '3 n': 'Yellow', '4 d': 'Black'}
{'0 c': 'Green', '1 a': 'Black', '2 c': 'Yellow', '3 t': 'Black', '4 i': 'Yellow'}
{'0 c': 'Green', '1 y': 'Green', '2 n': 'Green', '3 i': 'Green', '4 c': 'Green'}


5

### Solver  
(feel free to edit this part to give better solutions! maybe with better heuristics/ reinforcement learning etc.)

#### Logic:
First guess to be 'CRANE' (because of what 3Blue1Brown found. Can verify if it really is the best ourselves)

Version 1.0 (15/2):
First implementation - A 'strong guessing' approach (i.e. hard difficulty on Wordle), but we are going to optimize it slightly by first refering to the most common 5-letter words (we have a list of 1367 words) before going on to all 5-letter words (10,422 words)



#### 1. Get five-letter words from word list (US English dictionary)

In [3]:
import nltk
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
# get all 5 letter words
n = 5
wordle_words = []

for i in words.words():
    if len(i) == n:
        wordle_words.append(i.lower())
        
        
len(wordle_words)

10422

In [5]:
# Also get the most common 5 letter words
list_ = open("10000_most_common_words.txt").read().split()
common_5L_words = []

for i in list_:
    if len(i) == n:
        common_5L_words.append(i.lower())
        
len(common_5L_words)

1367

#### 2. Actual Solver class

In [216]:
import random

wordle_words = common_5L_words


class WordleSolver:
    def __init__(self, word):
        self.word = word
        self.number_of_guesses = 0
        self.game_end = 0
        self.guess = 'CRANE'
        self.possible = wordle_words
        self.no_of_guesses = 0
        
    def guess_generator(self, info): # This is the part to be edited if you wanna play around with your own logic!
        
        # Example info: {'0 c': 'Green', '1 r': 'Black', '2 a': 'Black', '3 n': 'Yellow', '4 e': 'Black'}
        lst = []
        cond_words = []
        possible_words = wordle_words       
        guess = self.guess
        
        for key, value in info.items():
            lst.append([key, value])        
        
        for k in range(5):
            for word in wordle_words:    
                if lst[k][1] == 'Green':
                    if word[k] == lst[k][0][2:]:
                        cond_words.append(word)                       
                if lst[k][1] == 'Yellow':
                    if lst[k][0][2:] in word and word[k] != lst[k][0][2:]:
                        cond_words.append(word)        
                if lst[k][1] == 'Black':        
                    if lst[k][0][2:] not in guess[:k]: 
                        # if it is green later
                        if lst[k][0][2:] in guess[k+1:]:
                            if lst[self.find(guess,lst[k][0][2:])[1]][1] == 'Green':
                                cond_words = wordle_words
                            else:
                                if lst[k][0][2:] not in word:
                                    cond_words.append(word)      
                        else:
                            if lst[k][0][2:] not in word:
                                cond_words.append(word)
                    else:
                        cond_words = wordle_words        
            
            possible_words = intersection(possible_words, cond_words)
            cond_words = []
            
        possible_words = intersection(possible_words, self.possible)    
        self.possible = possible_words
        self.guess = random.choice(possible_words)
        
    def guesser(self, guess):
    
        wrd = WordleGame(word)
        info = wrd.check_all(guess)
        
        if wrd.number_of_correct_letter_pos == 5:
            self.game_end = 1

        return info
    
    def implement_guesses(self, guess):
        while self.game_end != 1:
            print(self.guess.upper())
            info = self.guesser(self.guess)
            self.guess_generator(info)
            self.no_of_guesses += 1

    def intersection(self, lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    
    def find(self, s, ch):
        return [i for i, ltr in enumerate(s) if ltr == ch]
            

In [221]:
# test

word = 'touch'

solver = WordleSolver(word)
solver.implement_guesses(word)
solver.no_of_guesses

CRANE
DUTCH
TOUCH


3

In [169]:
guess = 'aboon'

guess[3:]

lst[self.find(guess,'e')[1]][1]

'on'

### Simulations

With full dictionary (with nonsense words): Ave 5.12 (over 50 iterations)

With dictionary of top 10,000 most common words: Ave 3.76 (over 200 iterations)

In [219]:
import random

n = 200
average = 0
total = 0

# randomly pick n words from the dictionary
res = [random.randrange(1, len(wordle_words), 1) for i in range(n)]

for i in res:
    word = wordle_words[i]
    print(word.lower())
    solver = WordleSolver(word)
    solver.implement_guesses(word)
    total += solver.no_of_guesses
    
average = total / n

average

shall
CRANE
GLASS
SHALL
steal
CRANE
TALES
STEAL
since
CRANE
UNCLE
FENCE
SINCE
shops
CRANE
BUDDY
HOWTO
SHOPS
verse
CRANE
WORSE
PURSE
VERSE
ocean
CRANE
OCEAN
clips
CRANE
CLOTH
CLIMB
CLICK
CLIFF
CLIPS
icons
CRANE
ICONS
abuse
CRANE
WASTE
ABUSE
fewer
CRANE
MERIT
FEVER
FEWER
brief
CRANE
TRIES
DRIED
BRIEF
teens
CRANE
PENNY
BEING
TEENS
fuzzy
CRANE
LLOYD
KITTY
PUPPY
FUZZY
error
CRANE
TRIES
ERROR
death
CRANE
BEATS
DEATH
every
CRANE
PEERS
QUERY
EVERY
layer
CRANE
FARES
PAPER
WATER
MAKER
LAYER
davis
CRANE
VAULT
DAVIS
ideas
CRANE
SPEAK
IDEAS
sleep
CRANE
LIKES
STEEL
SLEEP
stays
CRANE
STAFF
STAYS
elect
CRANE
EXCEL
ELECT
hopes
CRANE
TYPES
PIPES
HOPES
trick
CRANE
BRICK
TRICK
shown
CRANE
SONGS
SIMON
SHOWN
toner
CRANE
NIGER
TUNER
TONER
added
CRANE
TAPES
ADDED
samoa
CRANE
PAPUA
GAMMA
SAMBA
SAMOA
omaha
CRANE
THATS
IDAHO
OMAHA
dance
CRANE
DANCE
delay
CRANE
VEGAS
MEDAL
DELAY
those
CRANE
OXIDE
SMOKE
WHOSE
THOSE
suits
CRANE
SOUTH
SUITS
kevin
CRANE
HELEN
BEGIN
KEVIN
turns
CRANE
ROUND
TURNS
blood
CRANE
TIGHT
BLOO

3.76

### Give own input - this program will suggest the best next word to try

### --





### Archived Code (not most efficient but may be useful at some point)

In [ ]:
class WordleGame:
    def __init__(self, word):
        self.word = word.lower()
        self.correct_letters = []
        self.correct_letter_pos = {}
        self.number_of_correct_letters = 0
        self.number_of_correct_letter_pos = 0
        
    def check_right_letter(self, guess):
        for letter in guess:
            if letter in self.word:
                print(str(letter) + ' in position ' + str(guess.find(letter)) + ' is in word')
                self.correct_letters.append(letter)
        
    def check_right_position(self, guess):
        for i in range(5):
            if guess[i] == self.word[i]:
                self.correct_letter_pos[i] = guess[i]
                self.number_of_correct_letter_pos += 1
        print('Correct Letter and Position: ' + str(self.correct_letter_pos))
                
                
                
    def check_all(self, guess):
        guess = guess.lower()
        self.check_right_letter(guess)
        self.check_right_position(guess)
        
        if self.number_of_correct_letter_pos == 5:
            print('Word is ' + str(self.word).upper() + '. Game Complete!')
        

In [201]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

guess = 'bavin'
lst = [['0 a', 'Yellow'],
 ['1 b', 'Yellow'],
 ['2 o', 'Black'],
 ['3 o', 'Black'],
 ['4 n', 'Green']]


possible_words = wordle_words
cond_words = []


for k in range(5):
    for word in wordle_words:    
        if lst[k][1] == 'Green':
            if word[k] == lst[k][0][2:]:
                cond_words.append(word)                       
        if lst[k][1] == 'Yellow':
            if lst[k][0][2:] in word and word[k] != lst[k][0][2:]:
                cond_words.append(word)        
        if lst[k][1] == 'Black':        
            if lst[k][0][2:] not in guess[:k]: 
                # if it is green later
                if lst[k][0][2:] in guess[k+1:]:
                    if lst[find(guess,lst[k][0][2:])[1]][1] == 'Green':
                        cond_words = wordle_words
                else:
                    if lst[k][0][2:] not in word:
                        cond_words.append(word)
            else:
                cond_words = wordle_words
    possible_words = intersection(possible_words, cond_words)
    print(len(possible_words))
    cond_words = []
    
        
        
print(possible_words)

len(possible_words)

blah = possible_words


4196
534
438
438
34
['badan', 'bahan', 'bairn', 'bajan', 'baken', 'balan', 'basin', 'batan', 'bavin', 'befan', 'bejan', 'beman', 'besan', 'blain', 'brain', 'brawn', 'brian', 'bryan', 'bugan', 'buran', 'caban', 'cabin', 'cuban', 'imban', 'laban', 'leban', 'peban', 'rabin', 'reban', 'saban', 'urban', 'urban', 'basin', 'brain']


In [190]:
intersection(blah,blooh)

['badan',
 'bahan',
 'bajan',
 'balan',
 'basin',
 'batan',
 'bavin',
 'bugan',
 'imban',
 'laban',
 'saban',
 'basin']

In [197]:
len(blooh)

473

In [166]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


lst[find(guess,'e')[0]][1]

'Black'